In [299]:
import numpy as np
import pandas as pd
import plotly.express as px
import scipy
import scipy.stats

In [302]:
fname_rents = 'medianAskingRent_All.csv'
fname_zips = 'brooklyn_zip_neighborhood.csv'

In [303]:
rent_df = pd.read_csv(fname_rents)
rent_df['Borough'].unique()
rent_df = rent_df[rent_df['Borough']=='Brooklyn']
rent_df = rent_df.rename(columns={'areaName':'Neighborhood'})
zip_df = pd.read_csv(fname_zips)

In [304]:
double_neighborhoods = zip_df[zip_df['Neighborhood'].str.contains("/")]

In [305]:
lst0 = []
lst1 = []
for i in range (double_neighborhoods.shape[0]):
    
    d_neighborhoods = double_neighborhoods.iloc[i]['Neighborhood']
    print(i, d_neighborhoods)
    n_pair = d_neighborhoods.split("/")
    n_rows = rent_df.loc[rent_df['Neighborhood'].isin(n_pair)]
    if n_rows.shape[0] == 0:
        lst0.append(n_pair)
    elif n_rows.shape[0] == 1:
        lst1.append(n_pair)
    else:
        row_df = pd.DataFrame([[np.nan] * len(rent_df.columns)], columns=rent_df.columns)
        rent_df = rent_df.append(row_df, ignore_index=True)
        rent_df.iloc[-1:]['Neighborhood'] = d_neighborhoods
        rent_df.iloc[-1:]['Borough'] = n_rows.iloc[0]['Borough']
        rent_df.iloc[-1:]['areaType'] = n_rows.iloc[0]['areaType']
        rent_df.iloc[-1:,3:] = n_rows.iloc[:,3:].mean(axis=0)

    
#     print(n_pair)
#     print(n_rows.shape[0])
print(lst0)
print(lst1)

0 Brooklyn Heights/Cobble Hill
1 Parkville/Bensonhurst
2 Williamsburg/Bedford-Stuyvesant
3 Vanderveer/Flatbush
4 Bath Beach/Bensonhurst
5 Kensington/Windsor Terrace
6 Homecrest/Madison
7 Carroll Gardens/Red Hook
8 Sheepshead Bay/Brighton Beach
9 Brower Park/Crown Heights
10 Park Slope/Windsor Terrace
11 Park Slope/Gowanus
12 Bushwick/Bedford-Stuyvesant
13 Gravesend/Homecrest
14 Industry City/Sunset Park
15 Flatlands/Mill Basin
[['Homecrest', 'Madison']]
[['Parkville', 'Bensonhurst'], ['Vanderveer', 'Flatbush'], ['Brower Park', 'Crown Heights'], ['Gravesend', 'Homecrest'], ['Industry City', 'Sunset Park']]


In [306]:
rent_df.loc[rent_df['Neighborhood']=='Crown Heights','Neighborhood'] = 'Parkville/Bensonhurst'
rent_df.loc[rent_df['Neighborhood']=='Gravesend','Neighborhood'] = 'Gravesend/Homecrest'
rent_df.loc[rent_df['Neighborhood']=='Sunset Park','Neighborhood'] = 'Industry City/Sunset Park'
rent_df.loc[rent_df['Neighborhood']=='Bensonhurst','Neighborhood'] = 'Parkville/Bensonhurst'
rent_df.loc[rent_df['Neighborhood']=='Flatbush','Neighborhood'] = 'Vanderveer/Flatbush'

In [307]:
# axisvalues=list(range(1,len(rent_df.columns)-2))
# def calc_slope(row):
#     a = scipy.stats.linregress(row, y=axisvalues)
#     return a.slope 

# rent_df["Slope"]=rent_df.iloc[:,3:].apply(calc_slope,axis=1)
# rent_df.drop(["Slope"], axis = 1)

In [308]:
# rent_df.loc[rent_df['Neighborhood']=='Red Hook'].iloc[0,3:-1].plot.line()

In [309]:
rent_df.iloc[:,3:] = round(rent_df.iloc[:,3:].interpolate(axis=1),2)

In [310]:
# rent_df.iloc[:,3:].dropna(how="all")
cols = rent_df.columns[3:]
rent_df = rent_df.dropna(subset=cols,how="all")
# rent_df.iloc[:,3:] = rent_df.iloc[:,3:].dropna(how="all")

In [311]:
# rent_df

In [312]:
# rent_df.iloc[:,3:].apply(pd.Series.first_valid_index, axis=0)

In [313]:
rent_df.to_csv("rent_cleaned.csv")